# elec0135-assignment-cld

## Workings Notebook

I found it useful to work in Kaggle (given the 30 hours free per week of GPU time), then separate out the code into modules.

I've kept this notebook in the repo to show a track record of commits and for my own future reference.

### Kaggle Specific Code

In [29]:
# # Useful cleanups to reset status
# !rm -rf /kaggle/working/data
# !rm /kaggle/working/data.zip
# !rm results.csv
# !rm -rf /kaggle/working/artefacts

In [30]:
# also required in the `/interactive_runner.ipynb`
#!pip install gdown

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


### `/model/util.py`

In [40]:
"""
Functions for creating and training models, used across the various tasks.
"""
import keras
import numpy as np
import pandas as pd
import tensorflow as tf

from dataclasses import dataclass
from datetime import datetime, timedelta
from pathlib import Path
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.data import Dataset
from tensorflow.keras import layers, callbacks
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, Dense, Dropout, Flatten, GlobalAveragePooling2D, MaxPooling2D
from typing import NamedTuple, Tuple


class Params(NamedTuple):
    """
    Job Parameters Struct
    """
    image_size: int
    batch_size: int
    epochs: int
    epsilon: float
    early_stopping: bool
    early_stopping_patience: int
    adjust_learning_rate: bool
    opt: type
        
        
class ResultCollector():
    """
    Utility class to collect up and output results from tasks.
    """
    
    TRAIN_DETAILS_FILE = "train_details.csv"
    TEST_SCORES_FILE = "test_scores.csv"
    
    def __init__(
        self,
        path: Path
    ):
        self.path = path
        self.train_details = pd.DataFrame
        self.test_scores = pd.DataFrame
        
    def get_path(self) -> Path:
        return self.path

    def add_task_results(self, df_train, df_test) -> None:
        self.add_train_details(df_train)
        self.add_test_scores(df_test)
        
    def add_train_details(self, df: pd.DataFrame) -> None:
        if self.train_details.empty:
            self.train_details = df
        else:
            self.train_details = pd.concat([self.train_details, df])
        
        self._save(self.train_details, self.TRAIN_DETAILS_FILE)        

    def get_train_details(self) -> pd.DataFrame:
        return self.train_details
    
    def add_test_scores(self, df: pd.DataFrame) -> None:
        if self.test_scores.empty:
            self.test_scores = df
        else:
            self.test_scores = pd.concat([self.test_scores, df])
            
        self._save(self.test_scores, self.TEST_SCORES_FILE)
            
    def get_test_scores(self) -> pd.DataFrame:
        return self.test_scores
    
    def restore_results(self, quietly = True) -> None:
        try:
            self.train_details = pd.read_csv(self.path / self.TRAIN_DETAILS_FILE)
            self.test_scores = pd.read_csv(self.path / self.TEST_SCORES_FILE)
        except FileNotFoundError:
            print("Unable to restore history - starting fresh")
            if not quietly:
                raise
    
    def _save(self, df: pd.DataFrame, name: str) -> None:
        df.to_csv(self.path / name, index=False)


@dataclass
class ModelWrapper():
    """
    Utility class to hold the "outer" model, and the inner base model
    so that training can be fine-tuned if required.
    """    
    model: keras.Model
    base_model: keras.Model


def create_model(base_model_fn: str, params: Params, fc_layers = 2, fc_neurons = 1024, bn = False) -> ModelWrapper:
    """
    Create Keras application model, e.g.
        tf.keras.applications.EfficientNetV2B0
        tf.keras.applications.ConvNeXtBase
    with a custom top.
    """
    inputs = keras.Input(shape=(params.image_size, params.image_size, 3))
    # Base
    base_model = base_model_fn(weights='imagenet', include_top=False)
    base_model.trainable = False
    # set training=F here per https://keras.io/guides/transfer_learning/
    x = base_model(inputs, training=False)
    # Head
    x = GlobalAveragePooling2D()(x)
    if bn:
        x = BatchNormalization()(x)
    x = Flatten()(x)
    
    l = 0
    while (l < fc_layers):
        x = Dense(fc_neurons, activation="relu")(x)
        x = Dropout(0.5)(x)
        l = l + 1
    
    outputs = Dense(5, activation="softmax")(x)
    model = keras.Model(inputs, outputs)

    return ModelWrapper(model, base_model)


def run_task(task_id: str, model_wrapper: ModelWrapper,
             ds_train: Dataset, ds_valid: Dataset, ds_test: Dataset,
             params: Params, collector: ResultCollector, weights = None) -> Tuple[pd.DataFrame, pd.DataFrame]:
    
    model = model_wrapper.model
    # train
    start = datetime.datetime.now()
    df_train = train(task_id, model, ds_train, ds_valid, params)
    end = datetime.datetime.now()
    # test
    test_result = model.evaluate(ds_test)
    df_test = create_test_record(task_id, test_result, (end-start))
    # save CM too
    save_confusion_matrix(collector.get_path(), ds_test, model, task_id)
    return df_train, df_test


def train(task_id: str, model: Model,
             ds_train_: Dataset, ds_valid_: Dataset,
             params: Params, weights = None) -> pd.DataFrame:
    
    opt = params.opt
    print(f"Using: {opt}")
    model.compile(
        optimizer=params.opt(epsilon=params.epsilon),
        loss="categorical_crossentropy",
        metrics=['accuracy']
    )

    early_stopping = callbacks.EarlyStopping(
        min_delta=0.0001,
        patience=params.early_stopping_patience,
        restore_best_weights=True,
        verbose = 1
    )
    
    reduce_lr = callbacks.ReduceLROnPlateau(
        monitor = 'val_loss', factor = 0.3, 
        patience = 3, min_delta = 0.0005, 
        mode = 'min', verbose = 1)
    
    cbs = []
    if params.early_stopping:
        print("Using EarlyStopping")
        cbs += [early_stopping]
    if params.adjust_learning_rate:
        print("Using ReduceLROnPlateau")
        cbs += [reduce_lr]

    assert 1==2, "break"

    history = model.fit(
        ds_train_,
        validation_data=ds_valid_,
        epochs=params.epochs,
        verbose=1,
        callbacks=cbs,
        class_weight=weights
    )
   
    df_hist = pd.DataFrame(history.history)
    df_hist["task_id"] = task_id
    df_hist["epoch"] = df_hist.index
   
    return df_hist


def create_test_record(task_id: str, result: list[float], duration: timedelta):
    return pd.DataFrame({"task_id": [task_id], "test_loss" : [result[0]], "time_secs": [duration.seconds]})


def save_confusion_matrix(path: Path, ds: Dataset, model: Model, task_id: str) -> None:
    filepath = f"artefacts/conf_mat_{task_id}.png"
    filepath = path / filepath
    
    probabilities = model.predict(ds)
    predictions = np.argmax(probabilities, axis=1)

    one_hot_labels = np.concatenate([y for x, y in ds], axis=0)
    labels = [np.argmax(x) for x in one_hot_labels]
    
    result = confusion_matrix(labels, predictions, labels=[0,1,2,3,4], normalize='pred')
    disp = ConfusionMatrixDisplay(result, display_labels=[0,1,2,3,4])
    disp.plot()
    disp.ax_.set_title(task_id)
    
    print(f"Saving confusion matrix to {path}")
    disp.figure_.savefig(filepath, dpi=300)
    
    
def create_vgg_like_model(params: Params) -> ModelWrapper:
    inputs = keras.Input(shape=(params.image_size, params.image_size, 3))
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.25)(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.25)(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Dropout(0.25)(x)

    # classification layers
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)

    outputs = Dense(5, activation="softmax")(x)
    model = keras.Model(inputs, outputs)

    return ModelWrapper(model, None)



def create_simple_model(params: Params) -> Model:
    m = keras.Sequential([
        
        tf.keras.Input(shape=(params.image_size, params.image_size, 3)),
        
        # First Convolutional Block
        layers.Conv2D(filters=32, kernel_size=5, activation="relu", padding='same'),
        layers.Conv2D(filters=32, kernel_size=3, activation="relu", padding='same'),
        layers.MaxPool2D(),
        layers.Dropout(0.2),

        # Second Convolutional Block
        layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding='same'),
        layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding='same'),
        layers.MaxPool2D(),
        layers.Dropout(0.2),

        # Third Convolutional Block
        layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding='same'),
        layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding='same'),
        layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding='same'),
        layers.MaxPool2D(),
        layers.Dropout(0.2),

        # Classifier Head
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(units=5, activation="softmax"),
    ])
    return ModelWrapper(m, None)

### `/data/data_processing.py`

In [41]:
import gdown
import keras
import pandas as pd
import random
import shutil
import tensorflow as tf
import os
import zipfile

# handle different structure Kaggle (Notebook) vs. Colab (Modules)
# this wouldn't be kept in any "production" version.
try:
    from AMLS_II_assignment23_24.model.util import Params
except ModuleNotFoundError:
    pass

from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.data import Dataset
from tensorflow.data.experimental import AUTOTUNE
from tensorflow.keras import layers, callbacks
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image_dataset_from_directory
from typing import Tuple


def data_preprocessing(path: Path,
                       params: Params,
                       force=False) -> Tuple[Dataset, Dataset, Dataset, dict]:
    """
    """
    file = download_data(path, force)
    
    data_path = path / "data"
    if force:
        shutil.rmtree(data_path)
        
    if not data_path.exists():
        data_path.mkdir(parents=True, exist_ok=True)
       
        with zipfile.ZipFile(file, "r") as z:
            z.extractall(data_path)
        
    df_images = pd.read_csv((data_path / "train.csv"))
    
    imgs1 = random.sample(df_images[df_images.label==3].image_id.tolist(), k=2577)
    imgs2 = df_images[df_images.label!=3].image_id.tolist()
    
    df_images = df_images[df_images.image_id.isin((imgs1+imgs2))].copy()
    
    X_train, X_test, y_train, y_test = train_test_split(df_images.image_id, df_images.label, test_size=0.2, random_state=12)
    
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=12)
    
    train_path = create_ds_tree(X_train, y_train, data_path, "train")
    valid_path = create_ds_tree(X_valid, y_valid, data_path, "valid")
    test_path = create_ds_tree(X_test, y_test, data_path, "test")
    
    ds_train = create_dataset(train_path, params.image_size, params.batch_size)
    ds_valid = create_dataset(valid_path, params.image_size, params.batch_size)
    ds_test = create_dataset(test_path, params.image_size, params.batch_size, False)

    return ds_train, ds_valid, ds_test, extract_class_weights(df_images)


def download_data(path: Path, force=False) -> Path:
    """
    """
    url = "https://drive.google.com/uc?id=1TJBf1HZxAMpowZ92BcgS5N_NPHE7LPOT"
    output = path / "data.zip"
    if not Path(output).exists() or force:
        gdown.download(url, str(output), quiet=False)
    return output


def create_ds_tree(x, y, path: Path, name: str) -> Path:
    """
    Creates the directory structure for the given dataset.
    """
    ds_path = path / name
    if not ds_path.exists():
        ds_path.mkdir(parents=True, exist_ok=True)

        for lab in y.unique():
            (ds_path / str(lab)).mkdir(exist_ok=True)

        source_path = path / "train_images"
        
        for img, lab in zip(x, y):
            src = source_path / img
            dest = ds_path / str(lab) / img
            shutil.move(src, dest)
        
    return ds_path


def create_dataset(path: Path, img_size: int, batch_size: int, shuffle = True) -> Dataset:
    """
    """
    return image_dataset_from_directory(
        path,
        labels='inferred',
        label_mode='categorical',
        image_size=[img_size, img_size],
        batch_size=batch_size,
        seed=12345,
        shuffle=shuffle,
        crop_to_aspect_ratio=True
    )


def extract_class_weights(df_data: pd.DataFrame) -> dict:
    classes = df_data.label.unique()
    class_weights = compute_class_weight(class_weight='balanced',
                                         classes=classes,
                                         y=df_data.label)

    return dict(zip(classes, class_weights))


def convert_dataset(ds: Dataset) -> Dataset:
    """
    """
    def convert_to_float(image, label):
        image = tf.image.convert_image_dtype(image, dtype=tf.float32)
        image = image / 255.0
        return image, label

    return (
        ds
        .map(convert_to_float)
        .cache()
        .prefetch(buffer_size=AUTOTUNE)
    )
    

def augment_dataset_old(ds: Dataset, num_repeats: int) -> Dataset:
    """
    """
    def augment(image, label):
        seed = 12345
        image = tf.image.random_flip_left_right(image, seed)
        image = tf.image.random_flip_up_down(image, seed)
        image = tf.image.random_brightness(image, 0.2, seed)
        return image, label

    return (
        ds
        .repeat(num_repeats)
        .map(augment)
        .cache()
        .prefetch(buffer_size=AUTOTUNE)
    )


def augment_dataset(ds: Dataset, num_repeats: int) -> Dataset:
    """
    """
    def augment(image, label):
        seed = 12345
        image = tf.image.random_flip_left_right(image, seed)
        image = tf.image.random_flip_up_down(image, seed)
        image = tf.image.random_brightness(image, 0.2, seed)
        return image, label

    return (
        ds
        .repeat(num_repeats)
        .map(augment)
    )

def over_sample_class(ds: Dataset, class_label: int, batch_size: int, num_repeats: int = 1) -> Dataset:
    # filter dataset to just the class_label
    ds_filt = ds.unbatch().filter(lambda x, label: tf.equal(tf.argmax(label, axis=0), class_label))
    ds_filt = ds.repeat(num_repeats)
    # combined with original dataset, re-shuffle, and re-batch
    ds_over = tf.data.Dataset.concatenate(ds.unbatch(), ds_filt)
    ds_over = ds_over.shuffle(100000)
    ds_over = ds_over.batch(batch_size)
    return ds_over


### `/report.py`?

In [42]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_experiments_comp2(df_history: pd.DataFrame, task_ids: list, epoch_limit = 50) -> None:
    df = df_history[(df_history.task_id.isin(task_ids)) & (df_history.epoch <= epoch_limit)].copy()
    df["loss_gap"] = df.val_loss - df.loss
    df_grp = df[["epoch","task_id", "val_accuracy", "val_loss", "loss_gap"]].groupby(["epoch", "task_id"]).mean()
    fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(16, 8))
    sns.lineplot(data=df_grp, x="epoch", y="val_accuracy", hue="task_id",  ax=ax1)
    sns.lineplot(data=df_grp, x="epoch", y="val_loss", hue="task_id",  ax=ax2)
    sns.lineplot(data=df_grp, x="epoch", y="loss_gap", hue="task_id",  ax=ax3)

### `/main.py`

In [34]:
x = tf.keras.optimizers.Adam

In [35]:
type(x)

type

In [48]:
import datetime
import os
import pandas as pd
import tensorflow as tf

# handle different structure Kaggle (Notebook) vs. Colab (Modules)
# this wouldn't be kept in any "production" version.
try:
    from AMLS_II_assignment23_24.data_processing.pre_processing import data_preprocessing, rebatch
    from AMLS_II_assignment23_24.model import util as model_util
    from AMLS_II_assignment23_24.model.util import Params, ResultCollector, create_model
except ModuleNotFoundError:
    pass

from docopt import docopt
from pathlib import Path
from tensorflow.keras.optimizers import Adam, AdamW

tf.random.set_seed(67890)

# Starting set of params
params = Params(255, 196, 50, 0.005, True, 7, False, Adam)

ARTEFACTS_PATH = Path("artefacts")
ARTEFACTS_PATH.mkdir(parents=True, exist_ok=True)

collector = ResultCollector(ARTEFACTS_PATH)
collector.restore_results()

# Process Data
print("================")
print("= Loading Data =")
print("================")
ds_train, ds_valid, ds_test, class_weights = data_preprocessing(Path(os.getcwd()), params)
print(f"Class Weights: {class_weights}\n\n")

print("==== Task A: Explore Batch Size ====")
for bs in [64, 128, 192, 256]:
    print(f"Batch Size: {bs}")
    ds_train = ds_train.rebatch(bs)
    ds_valid = ds_valid.rebatch(bs)
    model = create_model(tf.keras.applications.ConvNeXtTiny, params) # can use default params here
    run_task(f"A_{bs}", model, ds_train, ds_valid, ds_test, params, collector)

# update based on results of Task A
params = Params(255, 256, 50, 0.005, True, 7, False, Adam)
ds_train, ds_valid, ds_test, class_weights = data_preprocessing(Path(cwd), params)

print("==== Task B: Explore Epsilon ====")
for e in [0.0025, 0.0050, 0.0075, 0.01]:
    print(f"Epsilon: {e}")
    p = Params(255, 256, 50, e, True, 7, False, Adam)    
    model = create_model(tf.keras.applications.ConvNeXtTiny, p)
    #run_task(f"B_{e}", model, ds_train, ds_valid, ds_test, collector, p)

# update based on results of Task B
params = Params(255, 256, 50, 0.0075, True, 7, False, Adam)
ds_train, ds_valid, ds_test, class_weights = data_preprocessing(Path(cwd), params)

print("==== Task C: Baseline Model Comparison ====")
for m in [tf.keras.applications.ConvNeXtTiny, tf.keras.applications.ConvNeXtBase,
          tf.keras.applications.EfficientNetB0, tf.keras.applications.EfficientNetV2B0]:
    print(f"Model: {m}")
    model = create_model(m, params)
    # run_task(f"C_{model.base_model.name}", model, ds_train, ds_valid, ds_test, collector, params)

print("==== Task D: Best-of-Breed Model ====")
params = Params(255, 256, 50, 0.0075, True, 7, False, AdamW)
# oversample & augment dataset
ds_train_aug = augment_dataset(over_sample_class(ds_train, 0, params.batch_size), 2)
# initial training
model_d = create_model(tf.keras.applications.EfficientNetV2B0, params, bn=True)
run_task(f"D_base", model_d, ds_train, ds_valid, ds_test, collector, params, class_weights)
# fine-tune by allowing base model to be re-trained
model_d.base_model.trainable = True
ft_params = Params(255, 256, 50, 1e-5, True, 5, False)
run_task(f"D_tuned", model, ds_train_aug, ds_valid, ds_test, collector, ft_params, class_weights)
del model_d

print("========================")
print("==== Ablation Study ====")
print("========================")

print("==== Task E: Remove Fine-Tuning ====")
model = create_model(tf.keras.applications.ConvNeXtBase, params, bn=True)
run_task(f"E", model, ds_train_aug, ds_valid, ds_test, collector, params, class_weights)
del model

print("==== Task F: Remove Class Weights ====")
model = create_model(tf.keras.applications.ConvNeXtBase, params, bn=True)
run_task(f"F", model, ds_train_aug, ds_valid, ds_test, collector, params)
del model

print("==== Task G: Remove Data Augmentation ====")
model = create_model(tf.keras.applications.ConvNeXtBase, params, bn=True)
run_task(f"G", model, ds_train, ds_valid, ds_test, collector, params, class_weights)
del model

print("==== Task H: Remove Batch Norm ====")
model = create_model(tf.keras.applications.ConvNeXtBase, params)
run_task(f"H", model, ds_train, ds_valid, ds_test, collector, params, class_weights)
del model

print("==== Task I: Regress to the Adam Optimiser ====")
params = Params(255, 256, 50, 0.0075, True, 7, False, Adam)
model = create_model(tf.keras.applications.ConvNeXtBase, params)
run_task(f"I", model, ds_train, ds_valid, ds_test, collector, params, class_weights)
del model

print("==== Task J: Remove a FC Layer ====")
params = Params(255, 256, 50, 0.0075, True, 7, False, Adam)
model = create_model(tf.keras.applications.ConvNeXtBase, params, 1)
run_task(f"J_1", model, ds_train, ds_valid, ds_test, collector, params, class_weights)
del model

print("==== Task J: Add a FC Layer ====")
params = Params(255, 256, 50, 0.0075, True, 7, False, Adam)
model = create_model(tf.keras.applications.ConvNeXtBase, params, 3)
run_task(f"J_3", model, ds_train, ds_valid, ds_test, collector, params, class_weights)
del model
    
#     df_train, df_test = run_task(f"A_{bs}", model, ds_train, ds_valid, ds_test, batch_size_params)
#     collector.add_task_results(df_train, df_test)
#     print(model.model.evaluate(ds_test))

# print("==== Task A: Baseline Model ====")

# model = create_convnext_base(DEFAULT_PARAMS)
# df_train, df_test = run_task("A_base", model, ds_train, ds_valid, ds_test, DEFAULT_PARAMS)
# collector.add_task_results(df_train, df_test)

# print("==== Task B: Baseline + Data Augmentation ====")
# {
#     """
#     Per task A, but with data augmentation.
#     """
#     ds_train_aug = augment_dataset(ds_train, 2)
#     model = create_convnext_base(DEFAULT_PARAMS)
#     df_train, df_test = run_task("B_base_aug", model, ds_train_aug, ds_valid, ds_test, DEFAULT_PARAMS)
#     collector.add_task_results(df_train, df_test)    
# }

# print("==== Task C: Baseline + Data Augmentation + Class Weights ====")
# model = create_convnext_base(DEFAULT_PARAMS)
# {
#     """
#     Per task B but, given the large class imbalance, class weight supplied.
#     """
#     ds_train_aug = augment_dataset(ds_train, 2)
#     df_train, df_test = run_task("C_base_aug_wgts", model, ds_train_aug, ds_valid, ds_test, DEFAULT_PARAMS, class_weights)
#     collector.add_task_results(df_train, df_test)    
# }

# print("==== Task D: Baseline + Data Augmentation + Class Weights + Fine Tune ====")
# {
#     """
#     Per task C but, given the large class imbalance, class weight supplied.
#     """
#     fine_tune_params = Params(50, 196, 1, 1e-5, True, 5, False)
#     print(fine_tune_params)
#     model.base_model.trainable = True
#     ds_train_aug = augment_dataset(ds_train, 2)
#     df_train, df_test = run_task("D_base_aug_wgts_ft", model, ds_train_aug, ds_valid, ds_test, DEFAULT_PARAMS, class_weights)
#     collector.add_task_results(df_train, df_test)    
# }


Params(image_size=255, batch_size=196, epochs=50, epsilon=0.005, early_stopping=True, early_stopping_patience=7, adjust_learning_rate=False, opt=<class 'keras.src.optimizers.adam.Adam'>)
= Loading Data =
Found 6489 files belonging to 5 classes.
Found 2163 files belonging to 5 classes.
Found 2164 files belonging to 5 classes.
Class Weights: {0: 1.990064397424103, 1: 0.9882137962539973, 2: 0.9066219614417435, 4: 0.8394256887854094, 3: 0.8394256887854094}


==== Task A: Explore Batch Size ====
Batch Size: 64
Using: <class 'keras.src.optimizers.adam.Adam'>
Using EarlyStopping


AssertionError: break

In [ ]:
m1 = create_model_ablations(tf.keras.applications.ConvNeXtTiny, "base", params, 2, 1024)
m1.model.summary()